# Explore the neighborhoods (wijk) of Amsterdam 
#### the Data for the city center is available in a CSV file

In [10]:
# read the postal codes with locations
import pandas as pd
df_Ams_1 = pd.read_csv('Ams_PC_Centrum.csv')
df_Ams_1.head()

,id,postcode,woonplaats,wijk,gemeente,provincie,netnummer,latitude,longitude,soort
0,1,1011,Amsterdam,"Lastage, Oosterdokseiland, Rapenburg, Uilenbur...",Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres
1,2,1012,Amsterdam,Burgwallen (Oudezijde en Nieuwezijde),Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres
2,3,1015,Amsterdam,"Noordwestelijke Grachtengordel, Noord-Jordaan,...",Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres
3,4,1016,Amsterdam,"Westelijke Grachtengordel, Zuid-Jordaan, Raamp...",Amsterdam,Noord-Holland,20,52.371147,4.883705,Adres
4,5,1017,Amsterdam,"Zuidelijke Grachtengordel, Leidsebuurt, Weteri...",Amsterdam,Noord-Holland,20,52.364185,4.890775,Adres06


## Lets show where the Neighborhoods in Amsterdam are located
### Ill Take Damrak (a central street) as center (in red) and the Neighborhoods (in blue)

In [3]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library    

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-2.9.2         |   py36h45558ae_0         613 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    pandas-1.0.3               |   py36h830a2c2_1        11.1 MB  conda-forge
    pysocks-1.7.1              |   py36h9f0ad1d_1          27 KB  conda-

In [9]:
# lets take Damrak, Amsterdam as the center
latitude = 52.373884
longitude = 4.893936
Ams_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map

# add a red circle marker to represent the City Center
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='City Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(Ams_map)

# add the Boroughs as blue circle markers
for lat, lng, label in zip(df_TN_1.latitude, df_TN_1.longitude, df_TN_1.wijk):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(Ams_map)

# Show the map
Ams_map

# Now we found the neighborhoods with their location (lat, long) lets see what the venues are

#### lets first prepare the modules libraries and FourSquare

In [11]:
import requests # library to handle requests
import numpy as np
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
CLIENT_ID = 'LCCWIWOHKRNX1L4CZDGEMYCMGH0U5ARSK2XJRJBFUCZAAUCM' # your Foursquare ID
CLIENT_SECRET = 'CRDGQLSRYTL4UCE4PDN3HJRF5HZGPAFUHVYJ1NJSO1D0SVLY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


#### Let's create a function to get all venues in a radius of 500 m for all the neighborhoods

In [12]:
# set the radius of the search and a limit on the results
LIMIT = 100
radius = 450

In [13]:
# Define a function to extract venues from fourSquare for all our neigborhoods
def getVenues(neighborhoods, latitudes, longitudes, radius=radius):
    
    venues_list=[]
    for name, lat, lng in zip(neighborhoods, latitudes, longitudes):
        print(name)
          
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return the relevant information for each venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    all_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    all_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(all_venues)

#### Lets find our venues

In [14]:
amsterdam_venues = getVenues(neighborhoods=df_Ams_1['wijk'],
                           latitudes=df_Ams_1['latitude'],
                           longitudes=df_Ams_1['longitude']
                           )


Lastage, Oosterdokseiland, Rapenburg, Uilenburg, Valkenburg, Waterlooplein
Burgwallen (Oudezijde en Nieuwezijde)
Noordwestelijke Grachtengordel, Noord-Jordaan, Marnixplein
Westelijke Grachtengordel, Zuid-Jordaan, Raamplein
Zuidelijke Grachtengordel, Leidsebuurt, Weteringschans, Frederiksplein, Rembrandtplein
Weesperbuurt, Plantage, Sarphatistraat, Kadijken, Oostelijke Eilanden, Czaar Peterbuurt


#### Let see what we got!

In [16]:
# Lets see the total, and an example
print(amsterdam_venues.shape)
amsterdam_venues.head()

(405, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Lastage, Oosterdokseiland, Rapenburg, Uilenbur...",52.372976,4.903957,Black Gold,52.371787,4.903056,Coffee Shop
1,"Lastage, Oosterdokseiland, Rapenburg, Uilenbur...",52.372976,4.903957,OCHA,52.374024,4.901683,Thai Restaurant
2,"Lastage, Oosterdokseiland, Rapenburg, Uilenbur...",52.372976,4.903957,Eetcafé Van Beeren,52.372044,4.902295,Gastropub
3,"Lastage, Oosterdokseiland, Rapenburg, Uilenbur...",52.372976,4.903957,HPS,52.371683,4.907673,Cocktail Bar
4,"Lastage, Oosterdokseiland, Rapenburg, Uilenbur...",52.372976,4.903957,Nieuwmarkt,52.372567,4.900321,Plaza


In [18]:
# Lets see per neighborhood
amsterdam_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Burgwallen (Oudezijde en Nieuwezijde),73,73,73,73,73,73
"Lastage, Oosterdokseiland, Rapenburg, Uilenburg, Valkenburg, Waterlooplein",40,40,40,40,40,40
"Noordwestelijke Grachtengordel, Noord-Jordaan, Marnixplein",100,100,100,100,100,100
"Weesperbuurt, Plantage, Sarphatistraat, Kadijken, Oostelijke Eilanden, Czaar Peterbuurt",40,40,40,40,40,40
"Westelijke Grachtengordel, Zuid-Jordaan, Raamplein",83,83,83,83,83,83
"Zuidelijke Grachtengordel, Leidsebuurt, Weteringschans, Frederiksplein, Rembrandtplein",69,69,69,69,69,69


In [19]:
# Amnd how many unique categories we have
print('There are {} uniques categories.'.format(len(amsterdam_venues['Venue Category'].unique())))

There are 131 uniques categories.


### Lets analyze all neighborhoods
#### First we need to code the venues in a way we can use it for clustering (onehot encoding)

In [36]:
# one hot encoding
amsteram_onehot = pd.get_dummies(amsterdam_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
amsteram_onehot['Neighborhood'] = amsterdam_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [amsteram_onehot.columns[-1]] + list(amsteram_onehot.columns[:-1])
amsteram_onehot = amsteram_onehot[fixed_columns]
print (amsteram_onehot.shape)
amsteram_onehot.head()

(405, 132)


,Neighborhood,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,"Lastage, Oosterdokseiland, Rapenburg, Uilenbur...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Lastage, Oosterdokseiland, Rapenburg, Uilenbur...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Lastage, Oosterdokseiland, Rapenburg, Uilenbur...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Lastage, Oosterdokseiland, Rapenburg, Uilenbur...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Lastage, Oosterdokseiland, Rapenburg, Uilenbur...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category ¶

In [26]:
amsterdam_grouped = amsteram_onehot.groupby('Neighborhood').mean().reset_index()
# Lets check the size
print (amsterdam_grouped.shape)
amsterdam_grouped

(6, 132)


,Neighborhood,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Burgwallen (Oudezijde en Nieuwezijde),0.000,0.013699,0.000000,0.013699,0.000000,0.000000,0.013699,0.000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00
1,"Lastage, Oosterdokseiland, Rapenburg, Uilenbur...",0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025,0.025,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00
2,"Noordwestelijke Grachtengordel, Noord-Jordaan,...",0.000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.010000,0.020000,0.000,0.00
3,"Weesperbuurt, Plantage, Sarphatistraat, Kadijk...",0.025,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.025,0.35
4,"Westelijke Grachtengordel, Zuid-Jordaan, Raamp...",0.000,0.000000,0.000000,0.024096,0.000000,0.012048,0.012048,0.000,0.000,...,0.012048,0.012048,0.000000,0.000000,0.000000,0.00,0.000000,0.048193,0.000,0.00
5,"Zuidelijke Grachtengordel, Leidsebuurt, Weteri...",0.000,0.000000,0.014493,0.014493,0.014493,0.014493,0.014493,0.000,0.000,...,0.014493,0.000000,0.014493,0.028986,0.014493,0.00,0.014493,0.014493,0.000,0.00


#### Lets print each neighborhood along with the top most common venues

In [27]:
num_top_venues = 10

for hood in amsterdam_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = amsterdam_grouped[amsterdam_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Burgwallen (Oudezijde en Nieuwezijde)----
                  venue  freq
0                 Hotel  0.08
1           Coffee Shop  0.07
2                   Bar  0.05
3          Cocktail Bar  0.04
4          Burger Joint  0.04
5            Restaurant  0.04
6                Museum  0.03
7    Chinese Restaurant  0.03
8  Marijuana Dispensary  0.03
9          Dessert Shop  0.03


----Lastage, Oosterdokseiland, Rapenburg, Uilenburg, Valkenburg, Waterlooplein----
                venue  freq
0              Hostel  0.08
1         Coffee Shop  0.08
2  Chinese Restaurant  0.05
3      Breakfast Spot  0.05
4               Hotel  0.05
5        Cocktail Bar  0.05
6       Grocery Store  0.05
7       Deli / Bodega  0.05
8   French Restaurant  0.02
9              Church  0.02


----Noordwestelijke Grachtengordel, Noord-Jordaan, Marnixplein----
                venue  freq
0                 Bar  0.13
1  Italian Restaurant  0.06
2                Café  0.05
3      Sandwich Place  0.04
4         Coffee Shop 

### Lets put that into a pandas dataframe sorted on venues

In [28]:
# lets write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [33]:
# Lets make the dataframe 

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
amsterdam_venues_sorted = pd.DataFrame(columns=columns)
amsterdam_venues_sorted['Neighborhood'] = amsterdam_grouped['Neighborhood']

for ind in np.arange(amsterdam_grouped.shape[0]):
    amsterdam_venues_sorted.iloc[ind, 1:] = return_most_common_venues(amsterdam_grouped.iloc[ind, :], num_top_venues)

amsterdam_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Burgwallen (Oudezijde en Nieuwezijde),Hotel,Coffee Shop,Bar,Restaurant,Burger Joint,Cocktail Bar,Beer Bar,Plaza,Steakhouse,Clothing Store
1,"Lastage, Oosterdokseiland, Rapenburg, Uilenbur...",Hostel,Coffee Shop,Hotel,Grocery Store,Chinese Restaurant,Breakfast Spot,Cocktail Bar,Deli / Bodega,Pub,History Museum
2,"Noordwestelijke Grachtengordel, Noord-Jordaan,...",Bar,Italian Restaurant,Café,Sandwich Place,Coffee Shop,Hotel,Tapas Restaurant,Clothing Store,Marijuana Dispensary,Seafood Restaurant
3,"Weesperbuurt, Plantage, Sarphatistraat, Kadijk...",Zoo Exhibit,Restaurant,History Museum,Café,Plaza,Planetarium,Museum,Zoo,Ice Cream Shop,Hotel
4,"Westelijke Grachtengordel, Zuid-Jordaan, Raamp...",Hotel,Bar,Yoga Studio,French Restaurant,Coffee Shop,Chocolate Shop,Café,Sandwich Place,Bakery,Deli / Bodega
5,"Zuidelijke Grachtengordel, Leidsebuurt, Weteri...",Italian Restaurant,Marijuana Dispensary,Clothing Store,Hotel,Restaurant,Steakhouse,Coffee Shop,Pastry Shop,Canal,Plaza


# Cluster Neighborhoods into 5 clusters based on the visual map 

In [37]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [39]:
# set number of clusters
kclusters = 5

amsterdam_grouped_clustering = amsterdam_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(amsterdam_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 0, 1, 0, 4], dtype=int32)

In [40]:
# Lets create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
amsterdam_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [43]:
amsterdam_merged = df_Ams_1

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
amsterdam_merged = amsterdam_merged.join(amsterdam_venues_sorted.set_index('Neighborhood'), on='wijk')

amsterdam_merged.head()

,id,postcode,woonplaats,wijk,gemeente,provincie,netnummer,latitude,longitude,soort,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,1011,Amsterdam,"Lastage, Oosterdokseiland, Rapenburg, Uilenbur...",Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres,...,Hostel,Coffee Shop,Hotel,Grocery Store,Chinese Restaurant,Breakfast Spot,Cocktail Bar,Deli / Bodega,Pub,History Museum
1,2,1012,Amsterdam,Burgwallen (Oudezijde en Nieuwezijde),Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres,...,Hotel,Coffee Shop,Bar,Restaurant,Burger Joint,Cocktail Bar,Beer Bar,Plaza,Steakhouse,Clothing Store
2,3,1015,Amsterdam,"Noordwestelijke Grachtengordel, Noord-Jordaan,...",Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres,...,Bar,Italian Restaurant,Café,Sandwich Place,Coffee Shop,Hotel,Tapas Restaurant,Clothing Store,Marijuana Dispensary,Seafood Restaurant
3,4,1016,Amsterdam,"Westelijke Grachtengordel, Zuid-Jordaan, Raamp...",Amsterdam,Noord-Holland,20,52.371147,4.883705,Adres,...,Hotel,Bar,Yoga Studio,French Restaurant,Coffee Shop,Chocolate Shop,Café,Sandwich Place,Bakery,Deli / Bodega
4,5,1017,Amsterdam,"Zuidelijke Grachtengordel, Leidsebuurt, Weteri...",Amsterdam,Noord-Holland,20,52.364185,4.890775,Adres06,...,Italian Restaurant,Marijuana Dispensary,Clothing Store,Hotel,Restaurant,Steakhouse,Coffee Shop,Pastry Shop,Canal,Plaza


### Lets see what the clusters look like

In [48]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map of the clusters ans lets take Damrak, Amsterdam as the center
latitude = 52.373884
longitude = 4.893936
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(amsterdam_merged['latitude'], amsterdam_merged['longitude'], amsterdam_merged['wijk'], amsterdam_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Lets examine each cluster and then we will be able to make a choice
#### Cluster 1

In [49]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 0, amsterdam_merged.columns[[1] + list(range(5, amsterdam_merged.shape[1]))]]

,postcode,provincie,netnummer,latitude,longitude,soort,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,1015,Noord-Holland,20,52.379093,4.885109,Adres,0,Bar,Italian Restaurant,Café,Sandwich Place,Coffee Shop,Hotel,Tapas Restaurant,Clothing Store,Marijuana Dispensary,Seafood Restaurant
3,1016,Noord-Holland,20,52.371147,4.883705,Adres,0,Hotel,Bar,Yoga Studio,French Restaurant,Coffee Shop,Chocolate Shop,Café,Sandwich Place,Bakery,Deli / Bodega


In [50]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 1, amsterdam_merged.columns[[1] + list(range(5, amsterdam_merged.shape[1]))]]

,postcode,provincie,netnummer,latitude,longitude,soort,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,1018,Noord-Holland,20,52.368387,4.918091,Adres,1,Zoo Exhibit,Restaurant,History Museum,Café,Plaza,Planetarium,Museum,Zoo,Ice Cream Shop,Hotel


In [51]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 2, amsterdam_merged.columns[[1] + list(range(5, amsterdam_merged.shape[1]))]]

,postcode,provincie,netnummer,latitude,longitude,soort,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1011,Noord-Holland,20,52.372976,4.903957,Adres,2,Hostel,Coffee Shop,Hotel,Grocery Store,Chinese Restaurant,Breakfast Spot,Cocktail Bar,Deli / Bodega,Pub,History Museum


In [52]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 3amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 0, amsterdam_merged.columns[[1] + list(range(5, amsterdam_merged.shape[1]))]], amsterdam_merged.columns[[1] + list(range(5, amsterdam_merged.shape[1]))]]

,postcode,provincie,netnummer,latitude,longitude,soort,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,1012,Noord-Holland,20,52.373386,4.894064,Adres,3,Hotel,Coffee Shop,Bar,Restaurant,Burger Joint,Cocktail Bar,Beer Bar,Plaza,Steakhouse,Clothing Store


In [53]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 4, amsterdam_merged.columns[[1] + list(range(5, amsterdam_merged.shape[1]))]]

,postcode,provincie,netnummer,latitude,longitude,soort,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,1017,Noord-Holland,20,52.364185,4.890775,Adres06,4,Italian Restaurant,Marijuana Dispensary,Clothing Store,Hotel,Restaurant,Steakhouse,Coffee Shop,Pastry Shop,Canal,Plaza


## Based on all previous investigation I would recommend a French restaurant in Noordwestelijke Grachtengordel, Noord-Jordaan, Marnixplein